In [ ]:
import os
import sys

In [ ]:
IN_COLAB='google.colab' in sys.modules

In [ ]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)
    
    !pip install transformers -q
    !pip install mlflow -q
    !pip install ujson -q
    !pip install faiss-gpu -q
    !pip install SentencePiece -q    
    
    if not os.path.exists("ColBERT-X"):
        !git clone https://github.com/hltcoe/ColBERT-X.git
            
    PYTHON="python3"
else:
    WORKING_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/trabalho_final/"
    
    PỲTHON="python3.8"

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.

In [ ]:
CLUEWEB22_FOLDER="clueweb22-pt"

In [ ]:
CLUEWEB_COLBERT_CORPUS_FILENAME_FORMAT="clueweb22_pt_colbertx_{:02}.tsv"
CLUEWEB_COLBERT_TO_CLUEWEB_ID_FILENAME_FORMAT="clueweb22_pt_colbertx_to_clueweb_id_{:02}.tsv"
CLUEWEB_COLBERT_CORPUS_PASSAGE_FORMAT="{}\t{}"

In [ ]:
import json
import glob
import tqdm
import re

In [ ]:
corpus = glob.glob(os.path.join(CLUEWEB22_FOLDER, "*.jsonl"))

print(corpus[0])

clueweb22-pt/corpus_windowed_v2_pt0000.jsonl


### Convert corpus to ColBERT-X format

In [ ]:
colbertx_to_clueweb_id = []

with open(corpus[0]) as input_file:
    with open(os.path.join(CLUEWEB22_FOLDER, CLUEWEB_COLBERT_CORPUS_FILENAME_FORMAT.format(1)), "w") as output_file :
        for i, line in enumerate(tqdm.tqdm(input_file)):
            passage = json.loads(line)

            if i > 0:
                output_file.write("\n")

            colbertx_to_clueweb_id.append(passage['id'])

            output_file.write(CLUEWEB_COLBERT_CORPUS_PASSAGE_FORMAT.format(i, 
                                                                           re.sub('[\r\n\t]', ' ', passage['contents'])))


22764703it [14:10, 26754.50it/s]


In [ ]:
with open(os.path.join(CLUEWEB22_FOLDER, CLUEWEB_COLBERT_TO_CLUEWEB_ID_FILENAME_FORMAT.format(1)), "w") as output_file:
    for i, clueweb_id in enumerate(colbertx_to_clueweb_id):
        if i > 0:
            output_file.write("\n")

        output_file.write("{}\t{}".format(i, clueweb_id))

### Generate the index

In [ ]:
os.chdir("ColBERT-X")

In [ ]:
!python3 -m xlmr_colbert.index --similarity l2 \
		--checkpoint ../colbertx_experiments/mMSMARCO-pt/train.py/fine_tune_012/checkpoints/colbert-20000.dnn \
		--index_root ../clueweb22-pt --index_name colbertx_index_pt0000 \
		--collection ../clueweb22-pt/clueweb22_pt_colbertx_01.tsv --doc_maxlen 480 --query_maxlen 32 --dim 64 --amp --bsize=512 --chunksize=24

2023-06-11 10:49:34.134039: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-11 10:49:34.187034: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-11 10:49:35.161400: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[Jun 11, 10:49:45] #> Creating directory /content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR/ColBERT-X/experiments/dirty/index.py/2023-06-11_10.49.36 




[Jun 11, 10:49:51] #> Creating directory /content/drive/MyDrive/unicamp/ia368v

In [ ]:
!pip install faiss-gpu -q

In [ ]:
!python3 -m xlmr_colbert.index_faiss --index_root ../clueweb22-pt --index_name colbertx_index_pt0000